In [70]:
import json
import os
import re
import urllib

import lxml.html
import requests

In [71]:
BASE_URL = 'https://www.fantasyflightgames.com'

In [72]:
product_response = requests.get(BASE_URL + '/en/products/')

games = json.loads(re.search(r'collection_data = (\[.+\])', product_response.text).groups()[0])

for game in games:
    game_response = requests.get(BASE_URL + game['url'])
    root = lxml.html.fromstring(game_response.text)
    for link in root.xpath('//a[@class="support-item"]'):
        game_name = game['url'].strip('/').split('/')[-1]
        fname = link.attrib['href'].split('/')[-1]
        ddir = os.path.join('data', game_name)
        os.makedirs(ddir, exist_ok=True)
        dfile = os.path.join(ddir, fname)
        if not os.path.isfile(dfile):
            print('Downloading: {} - {}'.format(game_name, fname))
            r = requests.get(link.attrib['href'])
            with open(dfile, 'wb') as o:
                o.write(r.content)

In [73]:
archive_response = requests.get(BASE_URL + '/en/more/product-document-archive/')
root = lxml.html.fromstring(archive_response.text)
main_content = root.xpath('//div[@id="minisite-content"]')[0]

In [74]:
for a in main_content.xpath('//a'):
    if a.attrib.has_key('href') and 'ffg_content' in a.attrib['href']:
        game_name = urllib.parse.unquote(a.attrib['href'].split('/ffg_content/')[1].split('/')[0]).replace(' ', '_').lower()
        fname = urllib.parse.unquote(a.attrib['href'].split('/')[-1]).replace(' ', '_')
        ddir = os.path.join('data', 'archive', game_name)
        os.makedirs(ddir, exist_ok=True)
        dfile = os.path.join(ddir, fname)
        if not os.path.isfile(dfile):
            print('Downloading: {} - {}'.format(game_name, fname))
            r = requests.get(a.attrib['href'])
            with open(dfile, 'wb') as o:
                o.write(r.content)